In [1]:
from PIL import Image
import os
import matplotlib.pyplot as plt
import pandas as pd
import glob
import numpy as np

In [2]:
def get_mask_path(tile_x, tile_y, mask_type):
    path = f"./data/sentinel-2a-tile-{tile_x}x-{tile_y}y/masks/{mask_type}-mask.png"
    return path

In [3]:
def load_image(tile_path):
    img = Image.open(tile_path)
    return img

In [4]:
def get_tile_pixels(img):
    pixels = img.load()
    return pixels

In [5]:
def plot_image(img):
    plt.imshow(img)

In [6]:
def is_in_mask(mask_pixels, pixel_x, pixel_y):
    if mask_pixels[pixel_y, pixel_x] == (0, 0, 0, 255):
        return True
    else:
        return False

In [7]:
def pixels_in_mask(tile_x, tile_y,mask_type):
    pixel_list = []
    
    mask_path = get_mask_path(tile_x, tile_y, mask_type)    
    mask_img = load_image(mask_path)    
    mask_pix = get_tile_pixels(mask_img)    
    mask_img_size = mask_img.size
    
    for pixel_x in range(0,mask_img.size[0]):
        for pixel_y in range(0,mask_img.size[1]):
            in_mask = is_in_mask(mask_pix, pixel_x, pixel_y)
            if in_mask:
                pixel_list.append([pixel_x,pixel_y])
    return pixel_list

In [8]:
def get_timeseries_image_paths(tile_x, tile_y, band):
    path = f"./data/sentinel-2a-tile-{tile_x}x-{tile_y}y/timeseries/{tile_x}-{tile_y}-{band}*.png"
    images = glob.glob(path)
    return images

In [9]:
def get_dates(timeseries_paths_list,band):
    dates = []
    for path in paths:
        dates.append((path.split(sep = band + "-")[1]).split(sep = ".")[0])
    return dates

In [10]:
def get_band_df(pixel_df,image_paths):
    for path_index in range(0,len(image_paths)):
        sugar_df = pixel_df.query("sugarcane==True").drop(columns='sugarcane')
        for band in bands:
            path = image_paths.loc[path_index,band]
            im = load_image(path)
            pix = get_tile_pixels(im)
            for index in sugar_df.index:
                x = sugar_df.loc[index,'x']
                y = sugar_df.loc[index,'y']
                if band == 'TCI':
                    rgb = pix[y,x]
                    sugar_df.loc[index,'r'] = rgb[0]
                    sugar_df.loc[index,'g'] = rgb[1]
                    sugar_df.loc[index,'b'] = rgb[2]
                else:
                    sugar_df.loc[index,band] = pix[y,x]
        sugar_df['NDVI'] = (sugar_df['B08'] - sugar_df['B04'])/(sugar_df['B08'] + sugar_df['B04'])
        sugar_df['NDII'] = (sugar_df['B08'] - sugar_df['B11'])/(sugar_df['B08'] + sugar_df['B11'])
        sugar_df.to_csv(str("./flat files/" + image_paths.loc[path_index,"date"] + ".csv"))

In [11]:
def get_N_and_water(flat_file_path,feature_file_path):
    timeseries_flat_files = os.listdir(flat_file_path)
    for file in timeseries_flat_files:
        data = pd.read_csv(str(flat_file_path + "/" + file))
        data['veg_index'] = (data['B12'] - data['B08'])/(data['B12'] + data['B08'])
        data['leaf_water_content'] = (data['NDII'] - min(data['NDII']))/(max(data['NDII']) - min(data['NDII']))
        for index in data.index:
            if data.loc[index,'veg_index']<-0.35:
                data.loc[index,'harvested'] = False
            else:
                data.loc[index,'harvested'] = True

            if data.loc[index,'harvested']==True:
                data.loc[index,'min_N'] = np.nan
                data.loc[index,'max_N'] = np.nan
                data.loc[index,'leaf_water_content'] = np.nan
            elif data.loc[index,'NDVI'] < 0.55:
                data.loc[index,'min_N'] = 135
                data.loc[index,'max_N'] = 150
            elif data.loc[index,'NDVI'] <= 0.6 and data.loc[index,'NDVI'] > 0.55:
                data.loc[index,'min_N'] = 120
                data.loc[index,'max_N'] = 135
            elif data.loc[index,'NDVI'] <= 0.65 and data.loc[index,'NDVI'] > 0.6:
                data.loc[index,'min_N'] = 105
                data.loc[index,'max_N'] = 120
            elif data.loc[index,'NDVI'] <= 0.75 and data.loc[index,'NDVI'] > 0.65:
                data.loc[index,'min_N'] = 90
                data.loc[index,'max_N'] = 105
            elif data.loc[index,'NDVI'] <= 0.85 and data.loc[index,'NDVI'] > 0.75:
                data.loc[index,'min_N'] = 75
                data.loc[index,'max_N'] = 90
            else:
                data.loc[index,'min_N'] = 60
                data.loc[index,'max_N'] = 75

        data = data[['x','y','harvested','min_N','max_N','leaf_water_content']]
        data.to_csv(feature_file_path + '/' + file)